<a href="https://colab.research.google.com/github/smshozab/Math-Expression-Tokenizer/blob/main/Final_final_TOA_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import re
import ipywidgets as widgets
from IPython.display import display, HTML, Image
import graphviz
import json

# Token types for lexical analysis
class TokenType:
    MATH_OPERATOR = "MATH_OPERATOR"
    NUMERIC_VALUE = "NUMERIC_VALUE"
    PARENTHESIS = "PARENTHESIS"
    SYMBOL = "SYMBOL"
    VARIABLE = "VARIABLE"

# Token structure for lexical analysis
class Token:
    def __init__(self, type, value):
        self.type = type
        self.value = value

# DFA structure for pattern matching
class DFA:
    def __init__(self, pattern):
        self.pattern = pattern
        self.transition_table = {}

        # Build transition table
        for state in range(len(pattern) + 1):
            for char in set(pattern):
                next_state = min(len(pattern), state + 1)
                while next_state > 0 and pattern[:next_state] != pattern[state - next_state + 1:state] + char:
                    next_state -= 1
                self.transition_table[(state, char)] = next_state

    def match(self, text):
        state = 0
        matches = []
        for i, char in enumerate(text):
            state = self.transition_table.get((state, char), 0)
            if state == len(self.pattern):
                matches.append(i - len(self.pattern) + 1)
        return matches

    def generate_diagram(self):
        dot = graphviz.Digraph(format='png')
        for state in range(len(self.pattern) + 1):
            dot.node(str(state), shape='circle', style='filled', fillcolor='lightblue2' if state == len(self.pattern) else 'white')
        for state, char in self.transition_table:
            dot.edge(str(state), str(self.transition_table[(state, char)]), label=char)
        return dot

    def get_transition_table_html(self):
        rows = []
        for state in range(len(self.pattern) + 1):
            row = [f"<td>{state}</td>"]
            for char in sorted(set(self.pattern)):
                next_state = self.transition_table.get((state, char), 0)
                row.append(f"<td>{next_state}</td>")
            rows.append("<tr>" + "".join(row) + "</tr>")
        header = "<tr><th>State</th>"
        for char in sorted(set(self.pattern)):
            header += f"<th>{char}</th>"
        header += "</tr>"
        return f"<table border='1' cellspacing='0' cellpadding='5'>{header}{''.join(rows)}</table>"

# Tokenize the input expression
def tokenize_expression(expression):
    tokens = []
    number_buffer = ""
    variable_buffer = ""

    for c in expression:
        if c.isspace():
            continue
        elif c.isdigit() or c == '.':
            if variable_buffer:
                tokens.append(Token(TokenType.SYMBOL, variable_buffer))
                variable_buffer = ""
            number_buffer += c
        elif c.isalpha():
            if number_buffer:
                tokens.append(Token(TokenType.NUMERIC_VALUE, number_buffer))
                number_buffer = ""
            variable_buffer += c
        else:
            if number_buffer:
                tokens.append(Token(TokenType.NUMERIC_VALUE, number_buffer))
                number_buffer = ""

            if variable_buffer:
                tokens.append(Token(TokenType.VARIABLE, variable_buffer))
                variable_buffer = ""

            if c in {'+', '-', '*', '/', '^'}:
                tokens.append(Token(TokenType.MATH_OPERATOR, c))
            elif c in {'(', ')'}:
                tokens.append(Token(TokenType.PARENTHESIS, c))
            else:
                tokens.append(Token(TokenType.SYMBOL, c))

    if number_buffer:
        tokens.append(Token(TokenType.NUMERIC_VALUE, number_buffer))

    if variable_buffer:
        tokens.append(Token(TokenType.VARIABLE, variable_buffer))

    return tokens

# Function to validate mathematical expression
def validate_expression(expression):
    try:
        # Check if the expression contains any variables
        if any(char.isalpha() for char in expression):
            return True
        else:
            # Attempt to evaluate the expression
            eval(expression)
            return True
    except:
        return False

# Function to perform lexical analysis on input expression
def perform_lexical_analysis(expression):
    if not validate_expression(expression):
        result_text.value = "<b style='color:red;'>Error: Invalid mathematical expression</b>"
        # Disable DFA diagram and transition table buttons
        display_dfa_button.disabled = True
        display_transition_button.disabled = True
        return

    # If expression is valid, enable DFA diagram and transition table buttons
    display_dfa_button.disabled = False
    display_transition_button.disabled = False

    tokens = tokenize_expression(expression)

    result = f"<b>Mathematical Expression:</b> {expression}<br><br>"
    result += "<b>Tokens:</b><br>"
    for token in tokens:
        result += f"<b>{token.type}:</b> {token.value}<br>"
    result_text.value = result

    # Evaluate the expression and display the result
    try:
        result_value = eval(expression)
        result_text.value += f"<br><br><b>Result:</b> {result_value}"
    except Exception as e:
        result_text.value += f"<br><br><b style='color:red;'>Error: {str(e)}</b>"

# Function to display DFA diagram
def display_dfa(button):
    pattern = expression_text.value.strip()
    dfa = DFA(pattern)
    display_dfa_heading = "<h1 style='text-align: center;'>DFA</h1>"
    display(HTML(display_dfa_heading))
    display(Image(data=dfa.generate_diagram().pipe(format='png')))

# Function to display transition table
def display_transition_table(button):
    pattern = expression_text.value.strip()
    dfa = DFA(pattern)
    display_transition_heading = "<h1 style='text-align: center;'>Transition Table</h1>"
    display(HTML(display_transition_heading))
    # Generate HTML table with interactive features and styling
    html_table = f"""
    <style>
        .transition-table {{
            border-collapse: collapse;
            font-family: Arial, sans-serif;
            font-size: 14px;
            width: 100%; /* Set table width to 100% of container */
            height: auto; /* Set table height to auto */
        }}
        .transition-table th, .transition-table td {{
            border: 1px solid #dddddd;
            padding: 12px; /* Increase cell padding for better spacing */
            text-align: center;
            background-color: #f2f2f2;
            transition: background-color 0.3s ease;
        }}
        .transition-table th {{
            background-color: #36b1d6;
            color: white;
        }}
        .transition-table td:hover {{
            background-color: #ddd;
        }}
        .transition-table td.clicked {{
            background-color: blue !important;
            animation: highlight 0.5s ease;
        }}
        @keyframes highlight {{
            0% {{ background-color: blue; }}
            100% {{ background-color: transparent; }}
        }}
    </style>
    <table class='transition-table'>
        <tr>
            <th>State</th>
    """

    for char in sorted(set(pattern)):
        html_table += f"<th>{char}</th>"
    html_table += "</tr>"

    for state in range(len(pattern) + 1):
        html_table += "<tr>"
        html_table += f"<td>{state}</td>"
        for char in sorted(set(pattern)):
            next_state = dfa.transition_table.get((state, char), 0)
            transition_details = json.dumps({'state': state, 'input': char, 'next_state': next_state})
            html_table += f"<td onmouseover='showTooltip(this, {transition_details})' onmouseout='hideTooltip()' onclick='highlightTransition({transition_details})'>{next_state}</td>"
        html_table += "</tr>"
    html_table += "</table>"

    # Display the HTML table
    display(HTML(html_table))

# JavaScript functions for interactive features
javascript_code = """
<script type="text/Javascript">
    var tooltip = document.createElement('div');
    tooltip.className = 'tooltip';
    document.body.appendChild(tooltip);

    function showTooltip(cell, details) {
        var rect = cell.getBoundingClientRect();
        tooltip.innerHTML = 'State: ' + details.state + '<br>Input: ' + details.input + '<br>Next State: ' + details.next_state;
        tooltip.style.display = 'block';
        tooltip.style.top = rect.top - tooltip.offsetHeight + 'px';
        tooltip.style.left = rect.left + (cell.offsetWidth / 2) - (tooltip.offsetWidth / 2) + 'px';
    }

    function hideTooltip() {
        tooltip.style.display = 'none';
    }

    function highlightTransition(details) {
        var state = details.state;
        var input = details.input;
        var next_state = details.next_state;

        var table = document.getElementsByTagName('table')[0];
        var rows = table.getElementsByTagName('tr');
        for (var i = 1; i < rows.length; i++) {
            var cells = rows[i].getElementsByTagName('td');
            for (var j = 1; j < cells.length; j++) {
                var cell = cells[j];
                var cell_state = parseInt(cell.innerHTML);
                if (cell_state == next_state && cell.innerHTML != '0') {
                    cell.style.backgroundColor = 'yellow';
                } else {
                    cell.style.backgroundColor = '';
                }
            }
        }
    }
</script>
"""

# GUI
style = """
<style>
    .custom-widget {
        background-color: #f2f2f2;
        padding: 10px;
        border-radius: 5px;
        margin-bottom: 10px;
        color: #000000;
    }
    .custom-button {
        background-color: #36b1d6;
        border: none;
        color: #ffffff;
        padding: 10px 20px;
        text-align: center;
        text-decoration: none;
        display: inline-block;
        font-size: 14px;
        margin: 4px 2px;
        cursor: pointer;
        border-radius: 5px;
        vertical-align: middle; /* Align text vertically */
        line-height: 20px; /* Adjust line height */
        transition: background-color 0.3s ease;
    }
    .custom-button:hover {
        background-color: #1e90ff;
    }
    .loading-spinner {
        border: 4px solid rgba(0, 0, 0, 0.1);
        border-left-color: #36b1d6;
        border-radius: 50%;
        width: 20px;
        height: 20px;
        animation: spin 1s linear infinite;
        margin-right: 10px;
        vertical-align: middle;
    }
    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }
</style>
"""
display(HTML(javascript_code))
display(HTML(style))

expression_text = widgets.Textarea(
    value='',
    placeholder='Enter a mathematical expression',
    description='',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

analyze_button = widgets.Button(description="Analyze", style={'description_width': 'initial'})
analyze_button.on_click(lambda b: perform_lexical_analysis(expression_text.value))

display_dfa_button = widgets.Button(description="Display DFA Diagram", style={'description_width': 'initial'})
display_dfa_button.on_click(display_dfa)

display_transition_button = widgets.Button(description="Display Transition Table", style={'description_width': 'initial'})
display_transition_button.on_click(display_transition_table)

result_text = widgets.HTML(
    value='',
    placeholder='Results will be displayed here',
    description='',
    disabled=False,
    layout=widgets.Layout(width='80%')
)

loading_spinner = widgets.HTML(value='<div class="loading-spinner"></div>', layout=widgets.Layout(visibility='hidden'))

# Large headings
lexical_analyzer_heading = "<h1 style='text-align: center;'>Lexical Analyzer</h1>"
expression_text.add_class("custom-widget")
analyze_button.add_class("custom-button")
display_dfa_button.add_class("custom-button")
display_transition_button.add_class("custom-button")
result_text.add_class("custom-widget")

# Displaying components
display(HTML(lexical_analyzer_heading))
widgets.VBox([widgets.HBox([expression_text, loading_spinner]), analyze_button, display_dfa_button, display_transition_button, result_text])
